In [1]:
import chess
from tqdm import tqdm
from project import Searcher, Position, initial, sunfish_move, sunfish_move_to_str

In [2]:
hist = [Position(initial, 0, (True, True), (True, True), 0, 0)]
time_limit = 1.2  # Seconds, Could do a node limit??
move, info = sunfish_move(Searcher(), hist, time_limit=time_limit,)

print(f'Move: {sunfish_move_to_str(move)}, Info: {info}')

Move: g1f3, Info: {'depth': 6, 'gamma': 20, 'score': 19, 'nodes': 30416}


In [3]:
hist[0].move(move)

Position(board='\n         \n         \nr.bkqbnr \npppppppp \n..n..... \n........ \n........ \n........ \nPPPPPPPP \nRNBKQBNR \n         \n         ', score=-37, wc=(True, True), bc=(True, True), ep=0, kp=0)

In [4]:
[el for el in hist[0].gen_moves()]

[Move(i=81, j=71, prom=''),
 Move(i=81, j=61, prom=''),
 Move(i=82, j=72, prom=''),
 Move(i=82, j=62, prom=''),
 Move(i=83, j=73, prom=''),
 Move(i=83, j=63, prom=''),
 Move(i=84, j=74, prom=''),
 Move(i=84, j=64, prom=''),
 Move(i=85, j=75, prom=''),
 Move(i=85, j=65, prom=''),
 Move(i=86, j=76, prom=''),
 Move(i=86, j=66, prom=''),
 Move(i=87, j=77, prom=''),
 Move(i=87, j=67, prom=''),
 Move(i=88, j=78, prom=''),
 Move(i=88, j=68, prom=''),
 Move(i=92, j=73, prom=''),
 Move(i=92, j=71, prom=''),
 Move(i=97, j=78, prom=''),
 Move(i=97, j=76, prom='')]

In [6]:
def play_on(board, n):
    from copy import copy
    board0 = copy(board)
    for i in tqdm(range(n)):
        moves = [el for el in board.generate_legal_moves()]
        if len(moves):
            board.push(moves[-1])
        else:
            board = board0


In [7]:
n = int(2e5)
play_on(chess.Board(), n)

100%|██████████| 200000/200000 [00:14<00:00, 13566.52it/s]


In [8]:
play_on(hist[0], n)

100%|██████████| 200000/200000 [00:10<00:00, 19054.06it/s]
